### Mac OS Protobuf Error Fixing Code Snipet

In [11]:
pip install protobuf==3.20

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 5.9 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.22.1
    Uninstalling protobuf-4.22.1:
      Successfully uninstalled protobuf-4.22.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-macos 2.11.0 requires protobuf<3.20,>=3.9.2, but you have protobuf 3.20.0 which is incompatible.
onnx 1.13.1 requires protobuf<4,>=3.20.2, but you have protobuf 3.20.0 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [6]:
import cv2
import mediapipe as mp
import numpy as np
import tensorflow as tf


NameError: name 'audio_classifier' is not defined

In [4]:
!brew reinstall protobuf

==> Fetching protobuf
==> Downloading https://ghcr.io/v2/homebrew/core/protobuf/manifests/21.12-2
######################################################################## 100.0%
==> Downloading https://ghcr.io/v2/homebrew/core/protobuf/blobs/sha256:09fdc2fcc
==> Downloading from https://pkg-containers.githubusercontent.com/ghcr1/blobs/sh
######################################################################## 100.0%
==> Reinstalling protobuf 
==> Pouring protobuf--21.12.arm64_ventura.bottle.2.tar.gz
🍺  /opt/homebrew/Cellar/protobuf/21.12: 288 files, 19.2MB
==> Running `brew cleanup protobuf`...
Disable this behaviour by setting HOMEBREW_NO_INSTALL_CLEANUP.
Hide these hints with HOMEBREW_NO_ENV_HINTS (see `man brew`).
==> No outdated dependents to upgrade!
==> Checking for dependents of upgraded formulae...
Disable this behaviour by setting HOMEBREW_NO_INSTALLED_DEPENDENTS_CHECK.
Hide these hints with HOMEBREW_NO_ENV_HINTS (see `man brew`).
==> Reinstalling 1 dependent with broken linka

In [3]:
!brew link --overwrite protobuf

To relink, run:
  brew unlink protobuf && brew link protobuf


In [1]:

# Load TFLite model
interpreter = tf.lite.Interpreter(model_path="/Volumes/T7/Project_Database/2023_Google_Solution_Challenge/DL-GoldenHour/wound_detection/wound_detection_model/best.tflite")
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Load MediaPipe Pose model
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

# Define function to find nearest landmark to a point
def find_nearest_landmark(point, landmarks):
    distances = np.linalg.norm(landmarks - point, axis=1)
    nearest_idx = np.argmin(distances)
    return landmarks[nearest_idx]

# Open webcam
cap = cv2.VideoCapture(0)

while True:
    # Read frame from webcam
    ret, frame = cap.read()

    # Run MediaPipe Pose model to get landmarks
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(image)
    landmarks = np.array([[landmark.x, landmark.y] for landmark in results.pose_landmarks.landmark])

    # Run TFLite model to predict wound location and name
    input_data = np.expand_dims(frame, axis=0)
    interpreter.set_tensor(input_details[0]['index'], input_data)
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])
    wound_location = output_data[0][:2]
    wound_name = output_data[0][2]

    # Draw red bounding box around wound location
    x, y = map(int, wound_location)
    cv2.rectangle(frame, (x-20, y-20), (x+20, y+20), (0, 0, 255), 2)

    # Find 2 landmarks near wound location
    nearest_landmarks = np.array([
        find_nearest_landmark(wound_location, landmarks),
        find_nearest_landmark(wound_location + np.array([0, -50]), landmarks)
    ])

    # If wound is a bite wound, find nearest landmark near shoulder or hip
    if wound_name == 'wound_ulcer':
        candidate_landmarks = np.array([
            find_nearest_landmark(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value], landmarks),
            find_nearest_landmark(landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value], landmarks),
            find_nearest_landmark(landmarks[mp_pose.PoseLandmark.LEFT_HIP.value], landmarks),
            find_nearest_landmark(landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value], landmarks)
        ])
        nearest_landmarks[1] = find_nearest_landmark(nearest_landmarks[0], candidate_landmarks)

    # Calculate midpoint between wound location and nearest landmark
    midpoint = np.mean(nearest_landmarks, axis=0)

    # Draw red circle at midpoint
    cv2.circle(frame, tuple(map(int, midpoint)), 10, (0, 0, 255), -1)

    # Show frame
    cv2.imshow('Frame', frame)

    # Exit if 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()


NameError: name 'tf' is not defined